# Notebook 22: Backend AI Integration - LangChain Patterns and FastAPI Endpoints

## 🎯 What You'll Learn

Now it's time to add AI superpowers to your FastAPI backend! In this notebook, you'll learn how to integrate **LangChain** with your existing Todo API to create two powerful AI features:

1. **📝 Text Summarization**: A general-purpose AI endpoint for summarizing any text
2. **🎭 Poem Generator**: Transform your todo items into creative poems using AI

**The best part?** Your existing todo CRUD operations will continue working exactly as before - we're just adding new AI-powered endpoints!

## 🔧 What We're Building

**New API Endpoints:**
- `POST /todos/summarize-text` - Summarize any text using AI
- `POST /todos/write-poem/{id}` - Generate a poem from a specific todo

**LangChain Integration:**
- Clean, organized AI code using modern patterns
- Reusable prompt templates
- Proper error handling for AI operations

---

**💡 Key Insight**: Adding AI to existing APIs is about enhancing, not replacing. Your core CRUD functionality stays intact while AI adds new capabilities.

## Part 1: Installing LangChain and Dependencies

### What We Need to Add

Your existing Todo app already has most of what we need. We just need to add a few AI-specific packages:

**Required Packages:**
- `langchain==0.1.1` - Main LangChain library
- `openai` - OpenAI API client (already included in LangChain)
- Your existing packages stay exactly the same!

### Installation Process

**Step 1: Navigate to Your Backend Directory**
```bash
cd your-todo-app/backend
```

**Step 2: Activate Your Virtual Environment**
```bash
# If using Poetry (modern todo app)
poetry shell

# If using venv (original todo app)
source venv/bin/activate  # macOS/Linux
# OR
venv\Scripts\activate     # Windows
```

**Step 3: Install LangChain**
```bash
# If using Poetry
poetry add langchain==0.1.1

# If using pip
pip install langchain==0.1.1
```

**Step 4: Update Requirements (if using pip)**
```bash
pip freeze > requirements.txt
```

### Why This Specific LangChain Version?

We're using **LangChain 0.1.1** because:
- ✅ **Stable and tested**: Known to work well with our patterns
- ✅ **Simple imports**: Uses the classic `from langchain import` syntax
- ✅ **Educational clarity**: Fewer abstractions to understand
- ✅ **Consistent results**: Same behavior across all installations

**⚠️ Important**: LangChain updates frequently and newer versions might have different syntax. Stick with 0.1.1 for this tutorial to avoid compatibility issues.

## Part 2: Environment Configuration - Adding OpenAI API Key

### Setting Up Your OpenAI API Key

**Step 1: Add to Environment File**

Open your `backend/.env` file and add your OpenAI API key:

```bash
# Existing environment variables (keep these!)
DATABASE_USER=todo_user
DATABASE_PASSWORD=todo_password
DATABASE_HOST=localhost
DATABASE_PORT=5432
DATABASE_NAME=modern_todo_db

# NEW: Add your OpenAI API key
OPENAI_API_KEY=sk-your-actual-api-key-here
```

**Step 2: Verify Environment Loading**

Your existing `config.py` should already handle environment variables. If you're using our modern setup, it looks like this:

```python
# config.py (should already exist)
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    database_user: str
    database_password: str
    database_host: str
    database_port: int
    database_name: str
    
    # Add this line for OpenAI
    openai_api_key: str = None  # Optional, with default
    
    model_config = ConfigDict(env_file=".env")
```

**Step 3: Environment Variable Security**

**✅ Good Practices:**
- Keep `.env` file in `.gitignore` (should already be there)
- Never commit API keys to version control
- Use different keys for development and production

**🔍 Check Your .gitignore:**
```
# Should already include
.env
__pycache__/
*.pyc
```

### How LangChain Will Find Your API Key

LangChain automatically looks for `OPENAI_API_KEY` in your environment variables. When you create an OpenAI instance:

```python
from langchain import OpenAI

# This automatically uses OPENAI_API_KEY from environment
llm = OpenAI(temperature=0)
```

**No need to explicitly pass the key!** LangChain handles this for you.

## Part 3: Understanding LangChain Components

Before we write code, let's understand the LangChain building blocks we'll use:

### 1. OpenAI LLM (Large Language Model)

```python
from langchain import OpenAI

# Create an OpenAI instance
llm = OpenAI(temperature=0)
```

**Parameters Explained:**
- **`temperature=0`**: More focused, consistent responses (range: 0-1)
  - `0` = very focused, predictable
  - `1` = very creative, unpredictable
  - `0.7` = balanced creativity (good for poems)

**What This Does:**
- Connects to OpenAI's API using your key
- Uses GPT-3.5-turbo by default (fast and cost-effective)
- Handles authentication automatically

### 2. PromptTemplate

```python
from langchain import PromptTemplate

# Create a reusable prompt template
prompt = PromptTemplate(
    template="Write a summary of: {text}",
    input_variables=['text']
)
```

**Why Use Templates?**
- **Consistency**: Same prompt format every time
- **Reusability**: Use with different inputs
- **Maintainability**: Change prompt in one place
- **Testing**: Easy to test different prompt versions

**Template Syntax:**
- `{variable_name}` gets replaced with actual values
- `input_variables` list must match template variables

### 3. LLMChain

```python
from langchain.chains import LLMChain

# Combine LLM + Prompt into a chain
chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# Use the chain
result = chain.run(text="Your input text here")
```

**What LLMChain Does:**
1. Takes your input variables
2. Fills in the prompt template
3. Sends the complete prompt to the LLM
4. Returns the AI's response

**Benefits of Chains:**
- **Clean code**: One line to run AI operations
- **Error handling**: Built-in retry and error management
- **Extensibility**: Can chain multiple operations together
- **Debugging**: Easy to see what's being sent to AI

## Part 4: Adding LangChain Imports to Your Existing Router

### Your Current todos.py File

Your existing `routers/todos.py` probably looks like this:

```python
# routers/todos.py (current)
from typing import List
from sqlalchemy.orm import Session
from fastapi import APIRouter, Depends, HTTPException, status
import schemas
import crud
from database import SessionLocal

router = APIRouter(prefix="/todos")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Your existing CRUD endpoints here...
@router.post("", status_code=status.HTTP_201_CREATED)
def create_todo(todo: schemas.TodoRequest, db: Session = Depends(get_db)):
    return crud.create_todo(db, todo)

# ... rest of your CRUD endpoints
```

### Adding LangChain Imports

**Step 1: Add New Imports**

Add these imports at the top of your `routers/todos.py` file:

```python
# routers/todos.py (enhanced)
from typing import List
from sqlalchemy.orm import Session
from fastapi import APIRouter, Depends, HTTPException, status
import schemas
import crud
from database import SessionLocal

# NEW: LangChain imports
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain

router = APIRouter(prefix="/todos")

# Your existing functions stay exactly the same!
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# All your existing CRUD endpoints continue working...
```

**Step 2: Test the Imports**

Start your server to make sure the imports work:

```bash
uvicorn main:app --reload
```

If you see any import errors:
- Make sure you installed `langchain==0.1.1`
- Make sure you're in the right virtual environment
- Try restarting your terminal and activating the environment again

**✅ Success Signs:**
- Server starts without errors
- Your existing endpoints work: `http://localhost:8000/docs`
- All your todo CRUD operations still function perfectly

**🚫 No Changes to Existing Code:**
- Your existing endpoints remain untouched
- Your database models stay the same
- Your CRUD functions continue working
- Your schemas don't change

We're adding AI on top of your existing, working system!

## Part 5: Creating the Text Summarization Feature

### Step 1: Set Up LangChain Components

Add this code to your `routers/todos.py` file, **after** your existing CRUD endpoints:

```python
# Add this AFTER your existing CRUD endpoints
# ==========================================
# LANGCHAIN AI FEATURES
# ==========================================

# Initialize OpenAI LLM
langchain_llm = OpenAI(temperature=0)

# Create prompt template for summarization
summarize_template_string = """
        Provide a summary for the following text:
        {text}
"""

summarize_prompt = PromptTemplate(
    template=summarize_template_string,
    input_variables=['text'],
)

# Create LLM chain for summarization
summarize_chain = LLMChain(
    llm=langchain_llm,
    prompt=summarize_prompt,
)
```

**Code Explanation:**

**`langchain_llm = OpenAI(temperature=0)`**
- Creates connection to OpenAI API
- `temperature=0` for consistent, focused summaries
- Uses your `OPENAI_API_KEY` automatically

**`summarize_template_string`**
- Clear, simple instruction to the AI
- `{text}` placeholder gets replaced with actual content
- Triple quotes for multi-line string (cleaner formatting)

**`summarize_prompt = PromptTemplate(...)`**
- Converts string template into LangChain object
- `input_variables=['text']` specifies what can be substituted
- Must match the `{text}` placeholder in template

**`summarize_chain = LLMChain(...)`**
- Combines the AI model with the prompt template
- Ready-to-use chain for summarization tasks
- Handles all the communication with OpenAI API

### Step 2: Create the API Endpoint

Add this endpoint **after** the LangChain setup code:

```python
# Text summarization endpoint
@router.post('/summarize-text')
async def summarize_text(text: str):
    try:
        summary = summarize_chain.run(text=text)
        return {'summary': summary}
    except Exception as e:
        raise HTTPException(
            status_code=500, 
            detail=f"AI summarization failed: {str(e)}"
        )
```

**Endpoint Explanation:**

**`@router.post('/summarize-text')`**
- Creates new endpoint: `POST /todos/summarize-text`
- Uses POST because we're sending data to be processed
- Part of the `/todos` router (your existing structure)

**`async def summarize_text(text: str):`**
- `async` because AI API calls can take time
- `text: str` parameter - FastAPI automatically parses request body
- Simple parameter type for easy testing

**`summary = summarize_chain.run(text=text)`**
- Uses our LangChain setup to process the text
- `run()` method executes the entire chain
- Returns the AI's summary as a string

**Error Handling:**
- `try/except` catches any AI API errors
- Returns proper HTTP error status (500)
- Provides helpful error message for debugging

## Part 6: Testing the Summarization Feature

### Step 1: Start Your Server

```bash
uvicorn main:app --reload
```

**✅ Check for Success:**
- Server starts without errors
- No import or syntax errors in the logs
- Your existing endpoints still work

### Step 2: Test with FastAPI Docs

1. **Open API Documentation**: `http://localhost:8000/docs`
2. **Find Your New Endpoint**: Look for `POST /todos/summarize-text`
3. **Click "Try it out"**
4. **Enter Test Text**:

```
The meeting covered several important topics. First, we discussed the quarterly budget and decided to increase the marketing spend by 15%. Second, we reviewed the hiring timeline for the engineering team and agreed to post the job listings next week. Finally, we set the deadline for the new product launch to be March 15th. The team expressed concerns about the tight timeline but committed to the goal. We also decided to have weekly check-ins to monitor progress.
```

5. **Click "Execute"**

**Expected Response:**
```json
{
  "summary": "The meeting discussed increasing marketing spend by 15%, posting engineering job listings next week, and setting a March 15th product launch deadline with weekly progress check-ins."
}
```

### Step 3: Test with Different Content

**Short Text:**
```
Buy groceries
```
*Expected: Brief or same text returned*

**Technical Text:**
```
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints. It provides automatic interactive API documentation, data validation, serialization, and authentication. The framework is built on top of Starlette for the web parts and Pydantic for the data parts.
```
*Expected: Concise summary of FastAPI's features*

### Step 4: Troubleshooting Common Issues

**❌ Error: "OpenAI API key not found"**
- Check your `.env` file has `OPENAI_API_KEY=sk-...`
- Restart your server after adding the key
- Make sure `.env` is in the `backend/` directory

**❌ Error: "Rate limit exceeded"**
- Wait a few seconds and try again
- Check your OpenAI dashboard for usage limits
- Make sure you have billing set up with OpenAI

**❌ Error: "Import error for langchain"**
- Make sure you installed `langchain==0.1.1`
- Check you're in the right virtual environment
- Try `pip list | grep langchain` to verify installation

**✅ Success Indicators:**
- API returns summarized text
- Response time is 2-5 seconds (AI processing takes time)
- Summary is shorter and more focused than original
- Your existing todo endpoints still work perfectly

## Part 7: Creating the Poem Generation Feature

### Step 1: Add Poem Generation Components

Add this code **after** your summarization setup:

```python
# Create prompt template for poem generation
write_poem_template_string = """
        Write a short poem with the following text:
        {text}
"""

write_poem_prompt = PromptTemplate(
    template=write_poem_template_string,
    input_variables=['text'],
)

# Create LLM chain for poem generation
write_poem_chain = LLMChain(
    llm=langchain_llm,  # Same LLM instance as summarization
    prompt=write_poem_prompt,
)
```

**Key Differences from Summarization:**

**Creative vs. Factual:**
- Poems are creative (we still use `temperature=0` for consistency)
- Summaries are factual and focused
- Same AI model, different prompt produces different behavior

**Reusing LLM Instance:**
- We use the same `langchain_llm` for both features
- Efficient resource usage
- Consistent API connection handling

### Step 2: Create the Poem Generation Endpoint

This endpoint will be more complex because it needs to:
1. Get a specific todo from the database
2. Use that todo's content to generate a poem
3. Handle cases where the todo doesn't exist

```python
# Poem generation endpoint
@router.post("/write-poem/{id}")
async def write_poem_by_id(id: int, db: Session = Depends(get_db)):
    try:
        # Get the todo from database
        todo = crud.read_todo(db, id)
        if todo is None:
            raise HTTPException(status_code=404, detail="Todo not found")
        
        # Generate poem from todo content
        poem = write_poem_chain.run(text=todo.name)
        return {'poem': poem}
        
    except HTTPException:
        # Re-raise HTTP exceptions (like 404)
        raise
    except Exception as e:
        # Handle AI API errors
        raise HTTPException(
            status_code=500, 
            detail=f"AI poem generation failed: {str(e)}"
        )
```

**Endpoint Explanation:**

**`@router.post("/write-poem/{id}")`**
- Creates: `POST /todos/write-poem/123`
- `{id}` is a path parameter for the todo ID
- Uses POST because it triggers AI processing

**`id: int, db: Session = Depends(get_db)`**
- `id: int` extracts todo ID from URL path
- `db: Session` injects database connection (existing pattern)
- Uses your existing `get_db()` function

**Database Integration:**
- `todo = crud.read_todo(db, id)` uses your existing CRUD function
- Follows your established patterns
- No new database code needed!

**Error Handling:**
1. **Todo not found** → 404 error (standard REST pattern)
2. **AI errors** → 500 error with helpful message
3. **Re-raise HTTP exceptions** → Preserves proper error responses

**AI Integration:**
- `poem = write_poem_chain.run(text=todo.name)` uses todo content
- Returns poem in consistent JSON format
- Integrates existing data with new AI capabilities

## Part 8: Testing the Poem Generation Feature

### Step 1: Make Sure You Have Todos

Before testing poem generation, create a few todos using your existing API:

1. **Open FastAPI Docs**: `http://localhost:8000/docs`
2. **Use `POST /todos/`** to create test todos:

**Test Todo 1:**
```json
{
  "name": "Study for math exam",
  "completed": false
}
```

**Test Todo 2:**
```json
{
  "name": "Finish painting project",
  "completed": false
}
```

**Test Todo 3:**
```json
{
  "name": "Buy groceries",
  "completed": false
}
```

3. **Use `GET /todos/`** to see your todos and note their IDs

### Step 2: Test Poem Generation

**Find Your New Endpoint**: `POST /todos/write-poem/{id}`

**Test with ID 1:**
1. Click "Try it out"
2. Enter `1` for the `id` parameter
3. Click "Execute"

**Expected Response:**
```json
{
  "poem": "Numbers dance across the page,\nFormulas waiting to be solved,\nKnowledge grows with every stage,\nAs mysteries get resolved.\n\nEquations hold the key to truth,\nMath exam approaching near,\nStudy hard and use your youth,\nSuccess will soon appear!"
}
```

### Step 3: Test Different Todo Content

**Creative Task (Painting):**
- Should generate artistic, colorful poems
- Words about creativity, colors, art

**Mundane Task (Groceries):**
- Should create something fun from ordinary content
- Shows AI's ability to find creativity in simple tasks

**Academic Task (Study):**
- Should generate motivational, focused poems
- Educational and encouraging tone

### Step 4: Test Error Scenarios

**Non-existent Todo ID:**
1. Try ID `999` (should not exist)
2. Expected response:
```json
{
  "detail": "Todo not found"
}
```
3. Status code should be `404`

**Invalid ID Format:**
1. Try ID `abc` (not a number)
2. FastAPI should return validation error
3. Shows built-in input validation working

### Step 5: Verify Integration Works

**✅ Success Checklist:**
- Poem endpoint finds existing todos correctly
- Generated poems relate to todo content
- Error handling works for missing todos
- Response format is consistent
- Your existing todo CRUD still works perfectly
- Both AI endpoints (summarize + poem) function

**📊 Performance Notes:**
- Poem generation takes 2-5 seconds (AI processing time)
- Database lookup is instant (uses existing optimized queries)
- Total response time is dominated by AI API call
- This is normal and expected for AI features!

## Part 9: Complete Code Example

Here's what your complete `routers/todos.py` file should look like with AI features added:

```python
from typing import List
from sqlalchemy.orm import Session
from fastapi import APIRouter, Depends, HTTPException, status
import schemas
import crud
from database import SessionLocal

# NEW: LangChain imports
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain

router = APIRouter(prefix="/todos")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# ==========================================
# EXISTING CRUD ENDPOINTS (unchanged)
# ==========================================

@router.post("", status_code=status.HTTP_201_CREATED)
def create_todo(todo: schemas.TodoRequest, db: Session = Depends(get_db)):
    return crud.create_todo(db, todo)

@router.get("", response_model=List[schemas.TodoResponse])
def get_todos(completed: bool = None, db: Session = Depends(get_db)):
    return crud.get_todos(db, completed=completed)

@router.get("/{todo_id}")
def get_todo_by_id(todo_id: int, db: Session = Depends(get_db)):
    todo = crud.get_todo(db, todo_id)
    if todo is None:
        raise HTTPException(status_code=404, detail="Todo not found")
    return todo

@router.put("/{todo_id}")
def update_todo(todo_id: int, todo: schemas.TodoRequest, db: Session = Depends(get_db)):
    updated_todo = crud.update_todo(db, todo_id, todo)
    if updated_todo is None:
        raise HTTPException(status_code=404, detail="Todo not found")
    return updated_todo

@router.delete("/{todo_id}")
def delete_todo(todo_id: int, db: Session = Depends(get_db)):
    deleted_todo = crud.delete_todo(db, todo_id)
    if deleted_todo is None:
        raise HTTPException(status_code=404, detail="Todo not found")
    return deleted_todo

# ==========================================
# NEW: LANGCHAIN AI FEATURES
# ==========================================

# Initialize OpenAI LLM
langchain_llm = OpenAI(temperature=0)

# Summarization setup
summarize_template_string = """
        Provide a summary for the following text:
        {text}
"""

summarize_prompt = PromptTemplate(
    template=summarize_template_string,
    input_variables=['text'],
)

summarize_chain = LLMChain(
    llm=langchain_llm,
    prompt=summarize_prompt,
)

# Poem generation setup
write_poem_template_string = """
        Write a short poem with the following text:
        {text}
"""

write_poem_prompt = PromptTemplate(
    template=write_poem_template_string,
    input_variables=['text'],
)

write_poem_chain = LLMChain(
    llm=langchain_llm,
    prompt=write_poem_prompt,
)

# AI Endpoints
@router.post('/summarize-text')
async def summarize_text(text: str):
    try:
        summary = summarize_chain.run(text=text)
        return {'summary': summary}
    except Exception as e:
        raise HTTPException(
            status_code=500, 
            detail=f"AI summarization failed: {str(e)}"
        )

@router.post("/write-poem/{id}")
async def write_poem_by_id(id: int, db: Session = Depends(get_db)):
    try:
        # Get the todo from database
        todo = crud.read_todo(db, id)
        if todo is None:
            raise HTTPException(status_code=404, detail="Todo not found")
        
        # Generate poem from todo content
        poem = write_poem_chain.run(text=todo.name)
        return {'poem': poem}
        
    except HTTPException:
        # Re-raise HTTP exceptions (like 404)
        raise
    except Exception as e:
        # Handle AI API errors
        raise HTTPException(
            status_code=500, 
            detail=f"AI poem generation failed: {str(e)}"
        )
```

### Key Architecture Points

**✅ Clean Separation:**
- Existing CRUD endpoints unchanged
- AI features added as separate section
- Both use same router and patterns

**✅ Resource Reuse:**
- Same `get_db()` dependency
- Same error handling patterns
- Same router prefix (`/todos`)

**✅ LangChain Best Practices:**
- Single LLM instance for multiple features
- Clear prompt templates
- Reusable chains for different tasks
- Proper error handling for AI operations

## 🎯 Key Takeaways

### What You've Accomplished:

1. **🔧 Added AI to Existing API**: Enhanced your todo app without breaking existing functionality
2. **⛓️ Learned LangChain Patterns**: PromptTemplate, LLMChain, and OpenAI integration
3. **🚀 Created Two AI Features**: Text summarization and creative poem generation
4. **💾 Integrated with Database**: AI features work with your existing todo data
5. **🛡️ Implemented Error Handling**: Robust AI integration with proper error responses
6. **📊 Maintained Performance**: Clean, efficient code using modern patterns

### LangChain Patterns You Can Reuse:

✅ **Single LLM, Multiple Chains**: One OpenAI instance for different AI tasks  
✅ **Template-Based Prompts**: Consistent, maintainable AI instructions  
✅ **Chain Pattern**: Reusable AI operations with proper error handling  
✅ **Database Integration**: Combine existing data with AI processing  
✅ **Progressive Enhancement**: Add AI without changing core functionality  

### Your New API Endpoints:

**📝 Text Summarization:**
- `POST /todos/summarize-text` - General-purpose text summarization
- Input: Any text string
- Output: AI-generated summary

**🎭 Poem Generation:**
- `POST /todos/write-poem/{id}` - Creative poems from todo content
- Input: Todo ID (uses existing database)
- Output: AI-generated poem based on todo text

### Next Steps:

In **Notebook 23**, we'll create the frontend integration:
- Add "Generate Poem" buttons to your React components
- Create popup displays for AI-generated content
- Handle loading states and error scenarios
- Build a complete AI-enhanced user experience

---

**🎉 Congratulations!** You've successfully added AI superpowers to your todo app backend. Your existing functionality is unchanged, but now you have powerful AI features that can enhance your users' experience. 

**Ready to bring these AI features to your users through the frontend? Let's build the React integration in Notebook 23! 🚀**